In [49]:
import requests
import os
from pprint import pprint
import pandas as pd

In [50]:
AIRTABLE_TOKEN: str = os.getenv("AIRTABLE_TOKEN")
DISCORD_TOKEN: str = os.getenv("DISCORD_TOKEN")
AIRTABLE_ID: str = os.getenv("AIRTABLE_ID")
AIRTABLE_PROJECT_TABLE: str = os.getenv("AIRTABLE_PROJECT_TABLE")
OPEN_AI_TOKEN: str = os.getenv("OPEN_AI_TOKEN")


In [51]:
def get_data(max_records: int = 100) -> dict:
    ENDPOINT = "https://api.airtable.com/v0"
    ID = "apphT7QAzpjDtBDHz"
    PROJECT_NAME = "Dum Dum Projects"
    
    params = {
        'maxRecords' : max_records
    }
    
    headers = {
        'Authorization' : f'Bearer {AIRTABLE_TOKEN}'
    }
    response = requests.get(
        f"{ENDPOINT}/{ID}/{PROJECT_NAME}",
        params=params,
        headers=headers
    )
    data = response.json()
    data = data['records']
    fields = [data['fields'] for data in data]
    df = pd.DataFrame(fields)[['Name', 'Description', 'Tags']]
    return df

In [52]:
def get_tags() -> list:
    df = get_data()
    tags = df['Tags'].tolist()
    tags = list(set([tag for tag_list in tags for tag in tag_list]))
    return tags

In [53]:
get_tags()

['Multimedia',
 'Research',
 'Flutter/Flutterflow',
 'Portfolio',
 'GenAI',
 'AI/ML',
 'Community',
 'Networking Systems',
 'Project Management',
 'Back-end',
 'Automation',
 'Product Creation',
 'Networking',
 'Financial',
 'Discord bots',
 'Diffusion Models',
 'NFC',
 'Design',
 'Front-end']

In [54]:
data = get_data(max_records=100)
data

,Name,Description,Tags
0,Culture center,NaN,"[Design, Networking, Community]"
1,Projects/startups agency,Dum dum projects-ish or like buildspace ish\n\...,"[Project Management, Community, Design]"
2,Bunbun,Bakery/Cafe,[Product Creation]
3,HuggingFace chat,NaN,"[AI/ML, Front-end, Back-end, GenAI, Diffusion ..."
4,Advent calendar or Festival box,NaN,"[Design, Product Creation]"
5,Caffeinated coffee bean,Podcast (or meows and mojitos or mochi and mocha),"[Community, Networking, Multimedia]"
6,VC,NaN,"[Project Management, Financial]"
7,Resume Tracker,Tracks your work portfolio/achievements and ke...,"[Front-end, Back-end, AI/ML, Discord bots]"
8,Brand collaboration agency,NaN,"[Project Management, Networking]"
9,Indogotochi,Indian collectible Gotochi Style postcard and ...,"[GenAI, Diffusion Models, Research, Design]"


# Assistant

In [55]:
# import assistant api from openai
import openai
from openai import AzureOpenAI

OPEN_AI_TOKEN = os.getenv("OPEN_AI_TOKEN")
client = openai.OpenAI(
    api_key=OPEN_AI_TOKEN
)

In [77]:
claire_instructions = """
Claire's Persona Prompt

You are Claire, the guild receptionist of an adventurer's guild. Your role is to handle adventurers' inquiries and tasks with professionalism and empathy, maintaining a friendly and approachable demeanor. You keep the guild lively by making small talk, sharing updates about what others are doing, and chatting casually while managing your responsibilities.

Personality : Claire is a warm and engaging receptionist who is always ready to help. She is professional, empathetic, and approachable, making adventurers feel welcome and supported. Claire is knowledgeable about the guild's projects and members, offering guidance and encouragement to those in need. She is japanese-european, with a friendly smile and a helpful attitude.
And she is vert attractive and witty, and flirty.

Guidelines:
Tone of Voice: Speak in a professional yet warm and engaging manner. Be friendly, empathetic, and relatable, while maintaining a polished tone.
Small Talk & Updates: Occasionally mention what other adventurers are working on or comment on guild happenings (data sourced from the Airtable API but presented as Claire “checking her records” or “consulting the job board”).
Scope Limitations: If a task or question is beyond your scope, politely direct the user to contact the Guild Master, Accelbia, for further assistance.
Example: "Hmm, this seems a bit out of my expertise. You might want to check with the Guild Master, Accelbia, for this one!"
Context Awareness: Understand the user's requests and provide thoughtful, guild-themed responses. Adapt to different levels of expertise and offer encouragement where needed.
Professional Empathy: If a task seems difficult, acknowledge the challenge while cheering them on.
Discretion on Airtable Usage: Present data retrieval as Claire referencing her guild records, job board, or personal notes.
Example: "Let me check my records… Ah, here we go~ This quest looks perfect for you!"
Backend Instructions:
Data Access: Retrieve information seamlessly from Airtable API to provide accurate project or guild updates.
Unclear Queries: Prompt users gently for clarification while maintaining a conversational tone.
Engagement: Mix small talk and updates naturally into interactions without overwhelming the user.
Fallback Responses: When a task exceeds Claire's capabilities, mention contacting the Guild Master, Accelbia.
Example: "Oh, this might need a bit of extra help. I'd recommend reaching out to the Guild Master, Accelbia. They'll know just what to do!"


Keep the responses in a format discord can understand. Keep them itemized and separated by new lines. Do not play with data to create new data. Unless asked for your own opinion over it, pass on the info as you receive it.


Function-Invoking Guides:
1. Retrieve Data: Use the Airtable API to fetch project information.
2. Generate Responses: Create engaging responses based on the retrieved data.
3. List Tags: Provide a list of tags to help users navigate the guild's projects.


If your reply requires any data from the Airtable API, please give the output "DEVCOM:<NUMBER>". Based on that you will get your new prompt with the data from the Airtable API.
Eg. 

User: Can you tell me about the projects?
Assistant: Sure! Here are the projects: \n DEVCOM:1

User : Can you tell me more about the project with the tag "AI"?
Assistant: Sure! Here are the projects with the tag "AI": \n DEVCOM:2
New Prompt: This is the data : <Data> \n Can you tell me more about the project with the tag "AI"?
Assistant: Sure! Here are the details of the project with the tag "AI": <Generated Response>

"""

In [82]:
def getResponse(prompt: str) -> str:
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "developer", "content": claire_instructions},
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    
    if "DEVCOM" in completion.choices[0].message.content:
        called_response = middleware(completion.choices[0].message.content, prompt)
        return called_response
    else:
        return completion.choices[0].message.content

In [87]:

def middleware(response: str, prompt: str) -> str:
    try: 
        if "DEVCOM:" in response:
            response = response.split("DEVCOM:")[1]
            response = int(response)
            if response == 1 or response == 2:
                projects = get_data()
                return getResponse(prompt + f" \n {projects}")
            
            if response == 3:
                tags = get_tags()
                return getResponse(prompt + f" \n {tags}")
    except Exception as e:
        return getResponse("Give an error response")
            

In [92]:
# Test the assistant
prompt = "I am not into coding stuff. Can you suggest me some projects?"

completion = getResponse(prompt)
print(completion)

Sure! Here are a few projects that aren't focused on coding:

1. Culture Center
   - **Tags:** Design, Networking, Community
   
2. Bunbun
   - **Description:** Bakery/Cafe
   - **Tags:** Product Creation

3. Advent Calendar or Festival Box
   - **Tags:** Design, Product Creation

4. Podcast (or meows and mojitos or mochi and mocha)
   - **Tags:** Community, Networking, Multimedia

5. Indian collectible Gotochi Style postcard and Hook
   - **Tags:** GenAI, Diffusion Models, Research, Design
   
6. Homemade Skincare
   - **Tags:** Product Creation

7. Small Town Book Club
   - **Tags:** Community, Discord bots

If any of these piques your interest, let me know! I’d be happy to share more details. 🌟
